# Random Walk Recommender

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/dm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/dm


## Import and Transform data 

In [3]:
import pandas as pd
import operator
import pickle
from scipy.sparse.linalg import gmres
from scipy.sparse import csr_matrix
import numpy as np
validation_books = pd.read_csv('validation.csv')
items = pd.read_csv('items.csv', sep='|')
transaction_original = pd.read_csv('transactions.csv', sep='|')
session_counts = transaction_original['sessionID'].value_counts()
transaction_original['session_count'] = transaction_original['sessionID'].apply(lambda x: session_counts[x])
print('there are %d transaction records in transactions' % transaction_original.shape[0])
print('there are %d recommended books in transactions' % len(set(validation_books['itemID']).intersection(set(
    transaction_original.index.values))))
transaction_original = transaction_original[transaction_original['session_count']>1]  # remove those transactions that only operate 1 book
recommended_books = set(validation_books['itemID']).intersection(set(transaction_original.index.values))
print('after removing transactions without only one session, there are %d records in transactions, %d recommend books out of 1000 books'
      % (transaction_original.shape[0], len(recommended_books)))
w_click = 0.1
w_basket = 0.3
w_order = 0.6
transaction_original['operate'] = (w_click * transaction_original['click'] + 
                                   w_basket * transaction_original['basket'] + 
                                   w_order * transaction_original['order'])
transaction_original = transaction_original.reset_index()
display(items.head())
display(validation_books.head())
display(transaction_original.head())

there are 365143 transaction records in transactions
there are 232 recommended books in transactions
after removing transactions without only one session, there are 129501 records in transactions, 98 recommend books out of 1000 books


,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"


,itemID,rec1_ID,rec2_ID,rec3_ID,rec4_ID,rec5_ID,rec6_ID,rec7_ID
0,55501,17214,NaN,NaN,NaN,NaN,NaN,NaN
1,50338,34534,NaN,NaN,NaN,NaN,NaN,NaN
2,23159,76833,58623,28475,75884,69228,NaN,20239
3,13382,8193,52254,78733,78225,20674,NaN,NaN
4,53793,63382,NaN,34773,76742,NaN,17001,23012


,index,sessionID,itemID,click,basket,order,session_count,operate
0,7,7,14576,1,1,0,2,0.4
1,8,7,17731,2,1,0,2,0.5
2,13,12,30277,1,0,0,3,0.1
3,14,12,29508,1,1,0,3,0.4
4,15,12,75659,1,0,0,3,0.1


In [9]:
def get_subtopics(items):
  num = dict()
  for j in range(len(items)):
    for i in str(items['subtopics'][j]).split(','):
      i = i.strip('[')
      i = i.strip(']')
      if i not in num:
        num[i] = 0
      num[i] += 1
  i = 0
  topic_list = []
  for key, value in num.items():
    if num[key] != 1:
      topic_list.append(key)
  return topic_list

In [10]:
topic_list = get_subtopics(items)

## Graph Construction

In [7]:
k_topic = 0.8
k_author = 1
k_sub = 0.5

def buildGrapha(items, transaction_original, k_topic, k_author, k_sub, topic_list):
  graph = dict()

  
  for i in range(len(items)):
    book,author,main_topic, sub_topics = 'book_' + str(items['itemID'][i]),str(items['author'][i]),str(items['main topic'][i]), str(items['subtopics'][i])
    if book not in graph:
      graph[book] = dict()
    graph[book][author] = k_author
    graph[book][main_topic] = k_topic
    if author not in graph:
      graph[author] = dict()
    graph[author][book] = k_author
    if main_topic not in graph:
      graph[main_topic] = dict()
    graph[main_topic][book] = k_topic
    for topic in sub_topics:
      topic = topic.strip('[')
      topic = topic.strip(']')
      if topic in topic_list:
        graph[book][topic] = k_sub
        if topic not in graph:
          graph[topic] = dict()
        graph[topic][book] = k_sub

  for i in range(len(transaction_original)):
    book, session, weight = 'book_' + str(transaction_original['itemID'][i]), 'session_' + str(transaction_original['sessionID'][i]), transaction_original['operate'][i]
    if book not in graph:
      graph[book] = dict()
    if session not in graph:
      graph[session] = dict()
    graph[book][session] = weight
    graph[session][book] = weight

  #transform the weight of edges between users and friends



  for point, point_dic in graph.items():
    weights_sum = sum(graph[point].values())   
    for next_point in graph[point]:
      graph[point][next_point] = graph[point][next_point] / weights_sum
  
  return graph

## Random Walk 

In [12]:
def random_walk(graph,root,alpha,iter_step,recom_num=10):
    rank={}
    rank={point:0 for point in graph}  
    rank[root]=1 
    result={}
    j = 1
    for i in range(iter_step):
        tmp_rank={}   
        tmp_rank={point:0 for point in graph}
        for out_point,out_dict in graph.items(): 
            for inner_point,value in graph[out_point].items():
                tmp_rank[inner_point]+=alpha*rank[out_point]*graph[out_point][inner_point]
                if inner_point==root:
                    tmp_rank[inner_point]+=1-alpha
        if tmp_rank==rank:
            print("converged")
            break  
        rank=tmp_rank
    right_num=0  
    for instance in sorted(rank.items(),key=operator.itemgetter(1),reverse=True):
        point,pr_score=instance[0],instance[1]
        if point == root:
          continue
        if point.split('_')[0] != 'book':
            continue  
        result[point]=pr_score
        right_num+=1
        if right_num>=recom_num:   
            break
    return result    

### Example

In [13]:
graph = buildGrapha(items, transaction_original, k_topic, k_author, k_sub, topic_list)
alpha=0.6
book='book_23159'
iter_num=10
recom_resul=random_walk(graph,book,alpha,iter_num,7)
print(recom_resul)

{'book_20239': 0.07805609961739159, 'book_65708': 0.07187372195129539, 'book_28475': 0.05473658084951303, 'book_26665': 0.04627077148549498, 'book_48941': 0.04209013456194485, 'book_31791': 0.03797367012073056, 'book_1766': 0.03794360619510502}


## Random Walk Based on Matrix

In [4]:
def get_matrix_from_graph(graph):
  M=[]
  for point in graph.keys():
      row = []
      for next_point in graph.keys():
          if next_point in graph[point]:
              weight=graph[point][next_point]
              row.append(weight)
          else:
              row.append(0)
      M.append(row)
  return np.matrix(M)

def get_csr_matrix(M, alpha):
  n = M.shape[0]
  M1 = np.eye(n) - alpha * M.T
  data = []
  row_list = []
  col_list = []
  for row in range(n):
    for col in range(n):
      if (M1[row, col] != 0):
        data.append(M1[row, col])
        row_list.append(row)
        col_list.append(col)
  M = csr_matrix((data, (row_list, col_list)), shape=(n, n))
  return M

def random_walk_based_on_matrix(graph, csr_matrix, book, alpha, recom_num=10):
  items=[]
  vertex=list(graph.keys())
  index=list(graph.keys()).index(book)
  n = csr_matrix.shape[0]
  zeros=np.zeros((n,1))
  zeros[index][0]=1
  r0=np.matrix(zeros)
  b = (1 - alpha) * r0
  res = gmres(csr_matrix, b, tol=1e-08)[0]
  tmp = {}
  result = {}
  for index in range(len(res)):
    point=vertex[index]
    tmp[point]=res[index]
  right_num=0  
  for instance in sorted(tmp.items(),key=operator.itemgetter(1),reverse=True):
      point,pr_score=instance[0],instance[1]
      if point == book:
          continue
      if point.split('_')[0] != 'book':
          continue   
      result[point]=pr_score
      right_num+=1
      if right_num>=recom_num:   
          break
  return result

In [ ]:
alpha = 0.8
M = get_matrix_from_graph(graph)
csr_matrix = get_csr_matrix(M, alpha)

In [ ]:
rc = {}
rc['book'] = []
rc['recom_book'] = []
rc['confidence'] = []
for i in range(len(validation_books)):
  book = 'book_' + str(validation_books['itemID'][i])
  result = random_walk_based_on_matrix(graph, csr_matrix, book, alpha, recom_num=10)
  for recom_book,confidence in result.items():
    rc['book'].append(book)
    recom_book = int(recom_book.split('_')[1])
    rc['recom_book'].append(recom_book)
    rc['confidence'].append(confidence)
df = pd.DataFrame(rc)
df.to_csv('results/recommendation_single_0.2.csv')